In [211]:
!pip install catboost

In [212]:
import numpy as np
import seaborn as sns #связано с pandas
import matplotlib.pyplot as plt
import pandas as pd

# kNN, логистическую регрессию, SVM, дерево решений, RandomForest, XGBoost, LightGBM и CatBoost

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import tree

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier


#ДЗ№4 Предобработка данных#

```
1. 
2. Survived: If the passenger survived(0-No, 1-Yes)
3. Pclass: Passenger Class (1 = 1", 2= 2", 3 = 3")
4. Sex: Male/Female
5. Age: Passenger age in years
6. SibSp: No of siblings/spouses aboard (сиблинги или супруги)
7. Parch: No of parents/children aboard (?? почему это называется иссушением/обжариванием? пресечене рода? о_О)
8. Fare: Passenger Fare (стоимость проезда)
9. Embarked: Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton) (порт отправления - 3 варианта)
--------------
// 10. Class - тоже, что pclass, но словами
// 11. Who - то же что и sex, но другими словами
// 12. Adult_male (bool) - объединение колонок возраста и пола
--------------
13. deck - буква палубы, где была каюта пассажира (ЕСТЬ NaNы) - A,B,C,D,E,F,G
----------------
// 14. embark_town - город погрузки полным словом (то же что embarked)
// 15. alive - то же что и survived (но словами)
----------------
16. alone (bool) - был ли человек в браке (?) -- очень похоже на 6 пункт (мб совпадает)
``` 
** **

**надо удалить 10, 11, 12, 14, 15**

проверить 16 на совпадение с 6

9 и 3 можно разбить на 3 булевские фичи



In [213]:
tnt = sns.load_dataset("titanic")
tnt

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


#**Удаляем столбцы-дубли**

In [214]:
tnt.drop(['class','who','adult_male', 'embark_town', 'alive'], axis='columns', inplace=True)
tnt

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck,alone
0,0,3,male,22.0,1,0,7.2500,S,NaN,False
1,1,1,female,38.0,1,0,71.2833,C,C,False
2,1,3,female,26.0,0,0,7.9250,S,NaN,True
3,1,1,female,35.0,1,0,53.1000,S,C,False
4,0,3,male,35.0,0,0,8.0500,S,NaN,True
...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,NaN,True
887,1,1,female,19.0,0,0,30.0000,S,B,True
888,0,3,female,NaN,1,2,23.4500,S,NaN,False
889,1,1,male,26.0,0,0,30.0000,C,C,True


#**Ищем и устраняем NaN**

In [215]:
print("В поле deck данные вида: ",tnt['deck'].unique())
print("В поле embarked данные вида: ",tnt['embarked'].unique())
print("В поле sex данные вида: ",tnt['sex'].unique())
print("В поле alone данные вида: ",tnt['alone'].unique())
print("--------------------------------")
print("Колонки, содержащие NaN: ",tnt.columns[tnt.isna().any()].tolist())
print("NaN-ов в Age: {} из {} ".format(tnt['age'].isna().sum(),tnt.shape[0])) # четверть данных - заменяем ср значением
print("NaN-ов в Embarked: {} из {} ".format(tnt['embarked'].isna().sum(),tnt.shape[0])) # оч мало - просто удаляем
print("NaN-ов в Deck: {} из {} ".format(tnt['deck'].isna().sum(),tnt.shape[0])) # большинство! - вводим новую палубу (Z - zero)

tnt = tnt.dropna(axis='index', how='any', subset=['embarked']) # удаляем 2 строки с nan в embarked

# нахожу средние значения в age согласно с alone и полом
#mean_age_m_alone = round(tnt[tnt['alone'] == 1][tnt['sex'] == 'male']["age"].mean())
#mean_age_fem_alone = round(tnt[tnt['alone'] == 1][tnt['sex'] == 'female']["age"].mean())

#mean_age_m_nalone = round(tnt[tnt['alone'] == 0][tnt['sex'] == 'male']["age"].mean())
#mean_age_fem_nalone = round(tnt[tnt['alone'] == 0][tnt['sex'] == 'female']["age"].mean())
#print(mean_age_m_alone, mean_age_fem_alone, mean_age_m_nalone, mean_age_fem_nalone)
mean_age = round(tnt['age'].mean())

# заменяем строки с nan в age на среднее (НЕ согласно с alone и полом)
tnt['age'] = tnt['age'].fillna(mean_age)
#tnt[tnt['alone'] == 1][tnt['sex'] == 'male']['age'] = tnt[tnt['alone'] == 1][tnt['sex'] == 'male']['age'].fillna(mean_age_m_alone)
#tnt['age'] = tnt[tnt['alone'] == 1][tnt['sex'] == 'female']["age"].fillna(mean_age_fem_alone)
#tnt['age'] = tnt[tnt['alone'] == 0][tnt['sex'] == 'female']["age"].fillna(mean_age_fem_nalone)
#tnt['age'] = tnt[tnt['alone'] == 0][tnt['sex'] == 'male']["age"].fillna(mean_age_m_nalone)
# более конкретная замена без inplace=true не работает, с inplace=true все делает nan-ами ПОЧЕМУ-ТО

tnt['deck'] = tnt['deck'].cat.add_categories('Z')
tnt['deck'] = tnt['deck'].fillna('Z') #вводим новую палубу (Z - zero) вместо NaN

print("после обработки NaN-ов в Age: {} из {} ".format(tnt['age'].isna().sum(),tnt.shape[0]))
print("после обработки NaN-ов в Embarked: {} из {} ".format(tnt['embarked'].isna().sum(),tnt.shape[0]))
print("после обработки NaN-ов в Deck: {} из {} ".format(tnt['deck'].isna().sum(),tnt.shape[0]))
print("Колонки, содержащие NaN после обработки: ",tnt.columns[tnt.isna().any()].tolist())


В поле deck данные вида:  [NaN, 'C', 'E', 'G', 'D', 'A', 'B', 'F']
Categories (7, object): ['C', 'E', 'G', 'D', 'A', 'B', 'F']
В поле embarked данные вида:  ['S' 'C' 'Q' nan]
В поле sex данные вида:  ['male' 'female']
В поле alone данные вида:  [False  True]
--------------------------------
Колонки, содержащие NaN:  ['age', 'embarked', 'deck']
NaN-ов в Age: 177 из 891 
NaN-ов в Embarked: 2 из 891 
NaN-ов в Deck: 688 из 891 
после обработки NaN-ов в Age: 0 из 889 
после обработки NaN-ов в Embarked: 0 из 889 
после обработки NaN-ов в Deck: 0 из 889 
Колонки, содержащие NaN после обработки:  []


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

**One-hot encoding** 

In [216]:
tnt = pd.get_dummies(tnt, columns = ['embarked']) # заменяем три порта отправки тремя колонками
tnt = pd.get_dummies(tnt, columns = ['deck']) # заменяем все палубы колонками

**Замена значений пола на 0 и 1 (male = 0, female = 1)**

In [217]:
tnt['sex'] = tnt['sex'].replace({'male': 0, 'female': 1})

**Нормализация fare**

In [218]:
tnt['fare']=(tnt['fare']-tnt['fare'].mean())/tnt['fare'].std()

#Тестирование

**Матрица корреляции**

In [219]:
corr = tnt.corr()
corr.style.background_gradient(cmap='coolwarm')

,survived,pclass,sex,age,sibsp,parch,fare,alone,embarked_C,embarked_Q,embarked_S,deck_A,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,deck_Z
survived,1.000000,-0.335549,0.541585,-0.075512,-0.034040,0.083151,0.255290,-0.206207,0.169966,0.004536,-0.151777,0.022701,0.166714,0.115629,0.151555,0.146142,0.058388,0.016261,-0.316101
pclass,-0.335549,1.000000,-0.127741,-0.326343,0.081656,0.016824,-0.548193,0.138553,-0.245733,0.220558,0.076466,-0.205968,-0.363033,-0.419177,-0.280167,-0.231420,0.010664,0.055478,0.720858
sex,0.541585,-0.127741,1.000000,-0.089927,0.116348,0.247508,0.179958,-0.306985,0.084520,0.075217,-0.121405,-0.078035,0.098987,0.059590,0.080015,0.047692,0.008591,0.091426,-0.137804
age,-0.075512,-0.326343,-0.089927,1.000000,-0.231691,-0.179377,0.087674,0.179114,0.034155,-0.009697,-0.023822,0.122834,0.077551,0.114119,0.132870,0.117714,-0.077192,-0.077611,-0.224275
sibsp,-0.034040,0.081656,0.116348,-0.231691,1.000000,0.414542,0.160887,-0.584186,-0.060074,-0.026692,0.069438,-0.046419,-0.030647,0.028976,-0.017789,-0.037082,0.001577,-0.001474,0.037447
parch,0.083151,0.016824,0.247508,-0.179377,0.414542,1.000000,0.217532,-0.583112,-0.011588,-0.081585,0.061512,-0.040476,0.062300,0.030462,-0.019340,-0.016764,0.023571,0.072335,-0.040445
fare,0.255290,-0.548193,0.179958,0.087674,0.160887,0.217532,1.000000,-0.274079,0.270731,-0.116684,-0.163758,0.019853,0.385381,0.365304,0.099410,0.054193,-0.032865,-0.025061,-0.481084
alone,-0.206207,0.138553,-0.306985,0.179114,-0.584186,-0.583112,-0.274079,1.000000,-0.094513,0.087099,0.027982,0.053042,-0.074223,-0.124678,-0.083371,-0.027849,-0.015762,-0.082648,0.164588
embarked_C,0.169966,-0.245733,0.084520,0.034155,-0.060074,-0.011588,0.270731,-0.094513,1.000000,-0.148646,-0.782613,0.092924,0.176892,0.113702,0.102796,-0.016153,-0.034868,-0.032452,-0.213044
embarked_Q,0.004536,0.220558,0.075217,-0.009697,-0.026692,-0.081585,-0.116684,0.087099,-0.148646,1.000000,-0.499261,-0.040342,-0.071105,-0.049970,-0.060463,-0.038036,-0.004197,-0.020703,0.128203


In [220]:
all = tnt[tnt.columns[1:]]
all

,pclass,sex,age,sibsp,parch,fare,alone,embarked_C,embarked_Q,embarked_S,deck_A,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,deck_Z
0,3,0,22.0,1,0,-0.499958,False,0,0,1,0,0,0,0,0,0,0,1
1,1,1,38.0,1,0,0.788503,False,1,0,0,0,0,1,0,0,0,0,0
2,3,1,26.0,0,0,-0.486376,True,0,0,1,0,0,0,0,0,0,0,1
3,1,1,35.0,1,0,0.422623,False,0,0,1,0,0,1,0,0,0,0,0
4,3,0,35.0,0,0,-0.483861,True,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2,0,27.0,0,0,-0.384258,True,0,0,1,0,0,0,0,0,0,0,1
887,1,1,19.0,0,0,-0.042189,True,0,0,1,0,1,0,0,0,0,0,0
888,3,1,30.0,1,2,-0.173986,False,0,0,1,0,0,0,0,0,0,0,1
889,1,0,26.0,0,0,-0.042189,True,1,0,0,0,0,1,0,0,0,0,0


# ДЗ №6 Ансамбль моделей ###

In [221]:
TRAIN_TEST_RATIO = 0.85 # доля экземпляров в тренировочном датасете по отношению к количеству всех экземпляров в исходном датасете

train_number = round(tnt.shape[0] * TRAIN_TEST_RATIO) # количество экземпляров в тренировочном датасете
alive = tnt['survived']
params = pd.DataFrame(tnt).drop(['survived'], axis = 1) 
#params = tnt[['age','alone']] # c params = tnt[['parch','pclass']] score = 0.7584269662921348

alive_train = np.array(alive).reshape((alive.shape[0], 1))[:train_number] # избегание ошибки с одномерным массивом
params_train = np.array(params).reshape((params.shape[0], tnt.shape[1] -1))[:train_number]

alive_test = np.array(alive).reshape((alive.shape[0], 1))[train_number:]
params_test = np.array(params).reshape((params.shape[0], tnt.shape[1] -1))[train_number:]

**kNN**

In [222]:
knn = KNeighborsClassifier()
knn.fit(params_train, alive_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

**SVM**

In [223]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', C = 0.1, gamma = 0.1)
alive_train = alive_train.astype('int')
classifier.fit(params_train[:,:], alive_train)

print("Score on test = {}".format(classifier.score(params_test, alive_test)))
print("Score on train = {}".format(classifier.score(params_train, alive_train)))

#result = alive_train.flatten() - classifier.predict(params_train[:,:])
#print("my Score on train = {}".format(result[result == 0].shape[0] / result.shape[0]))

Score on test = 0.706766917293233
Score on train = 0.6388888888888888


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [224]:
SVM = svm.LinearSVC()
SVM.fit(params_train, alive_train)

#print("Score in linearSVM = {}".format(SVM.score(all,alive)))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

## Логистическая регрессия

In [225]:
clf = LogisticRegression()
clf.fit(params_train, alive_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

##Дерево решений - c bagging## 

In [226]:
from sklearn.ensemble import BaggingClassifier
bagg = BaggingClassifier(tree.DecisionTreeClassifier(random_state=1))
bagg.fit(params_train, alive_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=1,
   

## Random Forest ##

In [227]:
from sklearn.ensemble import GradientBoostingClassifier
randomforest= GradientBoostingClassifier(learning_rate=0.01,random_state=1)
randomforest.fit(params_train, alive_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.01, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=1, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

## XGBoost ##

In [228]:
xgboostm=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
xgboostm.fit(params_train, alive_train)

print(xgboostm.predict(params_test))
print(alive_test.flatten())

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0
 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 0 1 0 0 0 1 0 0 1 1 0 0
 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0]
[0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0
 0 0 1 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 1 1
 0 0 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 1 1 1 1 0 0 0 1 0 0 1 1 0 0
 1 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0]


##LightGBM##

In [229]:
#define parameters
params = {'learning_rate':0.001}
alive_train1=alive_train.ravel()
train_data=lgb.Dataset(params_train,label = alive_train1)
lgbm = lgb.train(params, train_data, 100) 

pred_alive = lgbm.predict(params_test)

standardized_pred_alive = ((pred_alive - pred_alive.min()) / (pred_alive.max() - pred_alive.min())).astype(int)
#normalized_pred_alive = ((pred_alive - pred_alive.mean()) / pred_alive.std()).astype(int)

result = alive_test.flatten() - standardized_pred_alive
#print(alive_test.flatten())
#print(lgbm.best_score)
#standardized_pred_alive
score_lgb = result[result == 0].shape[0] / result.shape[0]

## CatBoost##

In [230]:
catbc = CatBoostClassifier()
categorical_features_indices = np.where(tnt.dtypes != np.float)[0]
# cat_features=([ 0, 2])
catbc.fit(params_train, alive_train, eval_set = (params_test, alive_test))

Learning rate set to 0.029574
0:	learn: 0.6742803	test: 0.6733529	best: 0.6733529 (0)	total: 1.4ms	remaining: 1.4s
1:	learn: 0.6579571	test: 0.6564398	best: 0.6564398 (1)	total: 2.9ms	remaining: 1.45s
2:	learn: 0.6414266	test: 0.6399068	best: 0.6399068 (2)	total: 4.16ms	remaining: 1.38s
3:	learn: 0.6270730	test: 0.6251054	best: 0.6251054 (3)	total: 5.44ms	remaining: 1.35s
4:	learn: 0.6143757	test: 0.6092401	best: 0.6092401 (4)	total: 6.73ms	remaining: 1.34s
5:	learn: 0.6033075	test: 0.5969109	best: 0.5969109 (5)	total: 7.92ms	remaining: 1.31s
6:	learn: 0.5919211	test: 0.5850328	best: 0.5850328 (6)	total: 9.2ms	remaining: 1.3s
7:	learn: 0.5788728	test: 0.5697583	best: 0.5697583 (7)	total: 10.5ms	remaining: 1.3s
8:	learn: 0.5685676	test: 0.5567280	best: 0.5567280 (8)	total: 11.7ms	remaining: 1.29s
9:	learn: 0.5601922	test: 0.5484919	best: 0.5484919 (9)	total: 13ms	remaining: 1.28s
10:	learn: 0.5497381	test: 0.5368811	best: 0.5368811 (10)	total: 14.2ms	remaining: 1.28s
11:	learn: 0.539068

## Таблица результатов ##

In [231]:
!pip install PrettyTable
from prettytable import PrettyTable  # Импортируем установленный модуль.

In [243]:
# Определяем твою шапку и данные.
th = ['','kNN', 'логистическая регрессия', 'SVM', 'дерево решений', 'RandomForest', 'XGBoost', 'LightGBM', 'CatBoost']
td = ['score',round(knn.score(params_test,alive_test),3),round(clf.score(params_test,alive_test),3),round(SVM.score(params_test,alive_test),3),
      round(bagg.score(params_test,alive_test),3),round(randomforest.score(params_test,alive_test),3), 
      round(xgboostm.score(params_test, alive_test),3), round(score_lgb,3), round(catbc.score(params_test, alive_test),3)]

columns = len(th)  # Подсчитаем кол-во столбцов на будущее.
table = PrettyTable(th)  # Определяем таблицу.

# Cкопируем список td, на случай если он будет использоваться в коде дальше.
td_data = td[:]
# Входим в цикл который заполняет нашу таблицу.
# Цикл будет выполняться до тех пор пока у нас не кончатся данные
# для заполнения строк таблицы (список td_data).
while td_data:
    # Используя срез добавляем первые n элементов в строку.
    table.add_row(td_data[:columns])
    # Используя срез переопределяем td_data так, чтобы он
    # больше не содержал первых n элементов.
    td_data = td_data[columns:]

print(table)  # Печатаем таблицу

+-------+-------+-------------------------+-------+----------------+--------------+---------+----------+----------+
|       |  kNN  | логистическая регрессия |  SVM  | дерево решений | RandomForest | XGBoost | LightGBM | CatBoost |
+-------+-------+-------------------------+-------+----------------+--------------+---------+----------+----------+
| score | 0.805 |          0.812          | 0.827 |      0.85      |     0.85     |  0.827  |  0.669   |  0.552   |
+-------+-------+-------------------------+-------+----------------+--------------+---------+----------+----------+


сделать другие метрики оценки

и разные параметры моделей в цикле

в catboost что-то с cat_features=([ 